## Preprocessing

In [1]:
import pandas as pd
import numpy as np
import glob
from pyntcloud import PyntCloud

In [2]:
def load_ply(file_name):
    cloud = PyntCloud.from_file(file_name)
    return cloud.points.values

In [3]:
def load_obj(file_name):
    vertices = []
    with open(file_name) as f:
        for line in f:
            if line[:2] == 'v ':
                index1 = line.find(' ') + 1
                index2 = line.find(' ', index1 + 1)
                index3 = line.find(' ', index2 + 1)

                vertex = (float(line[index1:index2]), float(line[index2:index3]), float(line[index3:-1]))
                vertex = [round(vertex[0], 2), round(vertex[1], 2), round(vertex[2], 2)]
                vertices.append(np.array(vertex))
                
    return np.array(vertices)

In [4]:
def load_off(file_name):
    vertices = []
    with open(file_name) as f:
        for i, line in enumerate(f):
            vals = line.split(' ')
            if i > 2 and len(vals) == 3:
                vertex = [float(vals[0]), float(vals[1]), float(vals[2])]
                vertices.append(np.array(vertex))

    return np.random.permutation(np.array(vertices))[:2048]

In [5]:
point_clouds = []
for i, file_name in enumerate(glob.glob('./data/04379243/*.ply')):
    
    points = load_ply(file_name)
    point_clouds.append(points)
    if i == 20:
        break
        
point_clouds = np.array(point_clouds, dtype='float64')

point_clouds.shape

(21, 2048, 3)

## Auto encoder

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [7]:
class Encoder(nn.Module):

    def __init__(self):
        super(Encoder, self).__init__()

        self.conv1 = nn.Conv1d(2048, 256, 1)
        self.conv2 = nn.Conv1d(256, 128, 1)
        self.conv3 = nn.Conv1d(128, 64, 1)
        self.conv4 = nn.Conv1d(64, 32, 1)
        self.maxPool1d = nn.MaxPool1d(3)


    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.maxPool1d(x)
        return x

encoder = Encoder()
print(encoder)

Encoder(
  (conv1): Conv1d(2048, 256, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
  (conv4): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
  (maxPool1d): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
)


In [8]:
class Decoder(nn.Module):

    def __init__(self):
        super(Decoder, self).__init__()

        self.fc1 = nn.Linear(32, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 2048*3)

    def forward(self, x):

        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        x = x.view(-1, 2048, 3)
        
        return x
    
decoder = Decoder()
print(decoder)

Decoder(
  (fc1): Linear(in_features=32, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=6144, bias=True)
)


In [9]:
class AutoEncoder(nn.Module):
    
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):

        x = self.encoder(x)
        x = self.decoder(x)
        
        return x

## Training

In [10]:
from lib.chamfer import ChamferLoss

model = AutoEncoder()

criterion = ChamferLoss()

num_epochs = 300
learning_rate = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

train_input = torch.from_numpy(point_clouds).float()

if torch.cuda.is_available:
    model = model.cuda()
    train_input = train_input.cuda()

for epoch in range(num_epochs):

    train_output = model(train_input)
    loss = criterion(train_output, train_input)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data[0]))

C:\tools\Anaconda2\envs\python36\lib\site-packages\ipykernel_launcher.py:26: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [10/300], loss:297.9619
epoch [20/300], loss:211.0951
epoch [30/300], loss:167.4910
epoch [40/300], loss:133.8176
epoch [50/300], loss:106.6925
epoch [60/300], loss:86.7010
epoch [70/300], loss:70.3970
epoch [80/300], loss:59.0366
epoch [90/300], loss:50.9145
epoch [100/300], loss:44.7963
epoch [110/300], loss:40.0339
epoch [120/300], loss:35.8852
epoch [130/300], loss:32.5109
epoch [140/300], loss:29.0754
epoch [150/300], loss:26.1767
epoch [160/300], loss:24.0825
epoch [170/300], loss:22.5180
epoch [180/300], loss:20.6729
epoch [190/300], loss:19.2295
epoch [200/300], loss:18.1551
epoch [210/300], loss:17.4493
epoch [220/300], loss:17.0104
epoch [230/300], loss:16.2729
epoch [240/300], loss:15.4895
epoch [250/300], loss:15.1692
epoch [260/300], loss:15.2019
epoch [270/300], loss:14.6596
epoch [280/300], loss:13.8604
epoch [290/300], loss:13.9948
epoch [300/300], loss:13.8393


#### save

In [ ]:
torch.save(encoder.state_dict(), './models/encoder.pt')
torch.save(decoder.state_dict(), './models/decoder.pt')

#### switch into cpu mode

In [ ]:
if torch.cuda.is_available :
    train_input  = train_input.cpu()  
    train_output = train_output.cpu()
    generator_output = generator_output.cpu()

## Generator

In [ ]:
class Generator(nn.Module):
    
    def __init__(self, encoder, decoder):
        super(Generator, self).__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):

        x = self.encoder(x)
        x[:,0,0] += 2
        x = self.decoder(x)
        
        return x

In [ ]:
g_encoder = Encoder()
g_decoder = Decoder()

g_encoder.load_state_dict(torch.load('./models/encoder.pt'))
g_decoder.load_state_dict(torch.load('./models/decoder.pt'))

generator = Generator(g_encoder, g_decoder)
generator_output = generator(train_input)

## Validation

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#### training data

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

index = 10

data = train_input[index].detach().numpy().T
ax.scatter(data[0], data[1], data[2], zdir='z', s=1, c='b', label='ground truth')

data = train_output[index].detach().numpy().T
ax.scatter(data[0], data[1], data[2], zdir='z', s=1, c='r', label='decoded')

ax.legend()
plt.show()

#### training (generator)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

index = 10

data = train_input[index].detach().numpy().T
ax.scatter(data[0], data[1], data[2], zdir='z', s=1, c='b', label='ground truth')

data = generator_output[index].detach().numpy().T
ax.scatter(data[0], data[1], data[2], zdir='z', s=1, c='r', label='decoded')

ax.legend()
plt.show()

## TODOs

* Create a mesh by marching cubes
* Manimulate latent vector or try something else to generate more nice images

## Misc

In [ ]:
input = torch.randn(10, 2048, 3)
print(input.shape)
m = nn.Conv1d(2048, 256, 1)
output = m(input)
print(output.shape)
m = nn.Conv1d(256, 128, 1)
output = m(output)
m = nn.Conv1d(128, 64, 1)
output = m(output)
print(output.shape)
m = nn.Conv1d(64, 32, 1)
output = m(output)
print(output.shape)
m = nn.MaxPool1d(3)
output = m(output)
print(output.shape)